# **NLP 02.텍스트 전처리 & 04.카운트 기반의 단어 표현 과제**
## TF-IDF를 이용한 와인 추천시스템 개발
본 과제는 딥러닝을 이용한 자연어처리 입문을 토대로 제작되었습니다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 KB 30.7 MB/s eta 0:00:00


In [4]:
# 필요한 라이브러리 import
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from nltk.tokenize import word_tokenize
from nltk. tokenize import WordPunctTokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from nltk.tokenize import sent_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
import re
from konlpy.tag import Okt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
# 데이터 불러오기
data = pd.read_csv('/content/drive/MyDrive/khuda/정규세션/심화트랙(NLP)/1주차 세션/wine reviews.csv의 사본')
data.head()

,Sl.No.,Brand,Name,Reviews Date Added,Reviews do Recommend,Reviews Num Helpful,Reviews Rating,Reviews Text,Reviews Title,Weight
0,1,Gallo,Ecco Domani174 Pinot Grigio - 750ml Bottle,2018-01-09T13:24:04Z,True,1.0,5.0,This a fantastic white wine for any occasion!,My Favorite White Wine,1.0 lbs
1,2,Fresh Craft Co.,Fresh Craft174 Mango Citrus - 4pk / 250ml Bottle,2018-01-09T17:31:52Z,True,NaN,5.0,"Tart, not sweet...very refreshing and delicious!",Yum!!,2.45 lbs
2,3,1000 Stories,1000 Stories174 Zinfandel - 750ml Bottle,2018-01-09T17:31:51Z,True,NaN,5.0,I was given this wine so it was a delightful s...,A New Favorite!,3.09 lbs
3,4,1000 Stories,1000 Stories174 Zinfandel - 750ml Bottle,2017-10-04T18:03:12Z,True,NaN,5.0,This is a phenomenal wine and my new favorite ...,"Bold, Flavorful, Aromatic, Delicious",3.09 lbs
4,5,Wine Cube153,Pink Moscato - 3l Bottle - Wine Cube153,2018-01-10T18:06:29Z,True,1.0,5.0,4 750ml bottles for the price of two With way ...,"Yum! Plus, Environmentally Friendly!",1.0 lbs


In [6]:
# data 확인
print(data.columns)
print(data.shape)

Index(['Sl.No.', 'Brand', 'Name', 'Reviews Date Added', 'Reviews do Recommend',
       'Reviews Num Helpful', 'Reviews Rating', 'Reviews Text',
       'Reviews Title', 'Weight'],
      dtype='object')
(2890, 10)


In [7]:
# data의 Reviews Text와 Name만 가져오기
data = data[['Name', 'Reviews Text']]
print('data size: ', data.shape)
display(data.head())

data size:  (2890, 2)


,Name,Reviews Text
0,Ecco Domani174 Pinot Grigio - 750ml Bottle,This a fantastic white wine for any occasion!
1,Fresh Craft174 Mango Citrus - 4pk / 250ml Bottle,"Tart, not sweet...very refreshing and delicious!"
2,1000 Stories174 Zinfandel - 750ml Bottle,I was given this wine so it was a delightful s...
3,1000 Stories174 Zinfandel - 750ml Bottle,This is a phenomenal wine and my new favorite ...
4,Pink Moscato - 3l Bottle - Wine Cube153,4 750ml bottles for the price of two With way ...


In [8]:
# 데이터 결측치 확인하기
data.isnull().sum()

Name            0
Reviews Text    1
dtype: int64

In [9]:
# 데이터 결측치 제거 후 결측치 다시 확인
data = data[data['Reviews Text'].notnull()].reset_index(drop=True)
data.isnull().sum()

Name            0
Reviews Text    0
dtype: int64

In [10]:
# 불용어 적용 후 'Reviews Text'에 대한 tf-idf 수행
tfidf = TfidfVectorizer(stop_words='english')#yourcode # 기본적으로 정의되어있는 불용어 적용

tfidf_matrix = tfidf.fit_transform(data['Reviews Text'])
print(tfidf_matrix.shape)

(2889, 5618)


In [11]:
# 코사인 유사도 계산하기
cosine_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
data = data.loc[:].reset_index(drop=True)

In [12]:
# wine name와 id를 매핑할 dictionary 생성
wine2id = {}
for i, c in enumerate(data['Name']): wine2id[i] = c

# id와 wine name를 매핑할 dictionary 생성
id2wine = {}
for i, c in wine2id.items(): id2wine[c] = i

In [13]:
# id2wine 딕셔너리 확인
id2wine

{'Ecco Domani174 Pinot Grigio - 750ml Bottle': 0,
 'Fresh Craft174 Mango Citrus - 4pk / 250ml Bottle': 1,
 '1000 Stories174 Zinfandel - 750ml Bottle': 3,
 'Pink Moscato - 3l Bottle - Wine Cube153': 4,
 "Beck's174 Non Alcoholic Beer - 6pk / 12oz Bottles": 5,
 'Apothic174 Red Blend - 750ml Bottle': 7,
 'California Roots Moscato White Wine - 750ml': 8,
 'Charles Charles174 Red Blend - 750ml Bottle': 9,
 'Fancy Pants174 Sauvignon Blanc - 750ml Bottle': 10,
 'Martini Rossi174 Asti Spumante Sparkling Wine - 750ml Bottle': 11,
 '603174 Winni Amber Ale - 6pk / 12oz Cans': 12,
 '603174 White Peaks Ipa - 6pk / 12oz Cans': 14,
 'Sofia Coppola174 Riesling - 750ml Bottle': 15,
 'California Roots Pinot Grigio White Wine - 750ml': 16,
 'Sancho Libre174 Peach - 4pk / 12oz Cans': 17,
 'Modern House174 With Love Red Blend - 750ml Bottle': 18,
 'Estancia174 Cabernet Sauvignon - 750ml Bottle': 19,
 'Voga Italia174 Pinot Grigio - 750ml Bottle': 20,
 'Michelob174 Ultra Light Cider Beer - 6pk / 12oz Bottles'

In [14]:
# 와인 추천시스템 함수 생성
def wine_recosys(wine_name):
  idx = id2wine[wine_name] 
  sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 와인들의 유사도 및 인덱스를 추출 
  sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
  sim_scores[0:10] # 상위 10개의 인덱스와 유사도를 추출
  sim_scores = [(wine2id[i], score) for i, score in sim_scores[0:10] if i in wine2id]
  return sim_scores #your code

In [15]:
 # 위의 딕셔너리에서 마음에 드는 와인 이름을 wine_recosys 함수에 입력해보세요.
 # 단, string 형태로 입력해야 합니다.
wine_recosys('Hornitos Reposado Tequila, 1.0 L')

[('Hornitos Reposado Tequila - 750ml', 1.0000000000000002),
 ('Sauza Hornitos Reposado Tequila, 1.75 L', 1.0000000000000002),
 ('Hornitos Reposado Tequila, 375 mL', 1.0000000000000002),
 ('Knob Creek Kentucky Straight Bourbon Whiskey, 1.75 L', 0.7120009813752735),
 ('Kinky174 Fruit Liqueur - 750ml Bottle', 0.536883484024857),
 ('Powell _ Mahoney Old Ballycastle Ginger Vintage Original Cocktail Mixer - 750 ml',
  0.41491550999274873),
 ('Zing Zang Bloody Mary Mix, 32 fl oz', 0.3827522642981839),
 ('Zing Zang Bloody Mary Mix, 32 fl oz', 0.3827522642981839),
 ('Knob Creek Kentucky Straight Bourbon Whiskey, 1.75 L', 0.3099535479587766),
 ('Master of Mixes Mango Daiquiri Margarita Mixer, 33.8 fl oz',
  0.24352306977838606)]

## 추가 과제

In [16]:
text = data['Reviews Text'][22]

In [17]:
# 영어 단어 토큰화
print(text)
print('nltk의 word_tokenize 토큰화: ', word_tokenize(text))#your code)
print('nltk의 WordPunctTokenizer 토큰화: ', WordPunctTokenizer().tokenize(text))#your code)
print('keras의 text_to_word_sequence 토큰화: ',text_to_word_sequence(text)) #your code)

Best of the bunch of all of The wine Cube sku's. Target Wine Buyer: please bring back!
nltk의 word_tokenize 토큰화:  ['Best', 'of', 'the', 'bunch', 'of', 'all', 'of', 'The', 'wine', 'Cube', 'sku', "'s", '.', 'Target', 'Wine', 'Buyer', ':', 'please', 'bring', 'back', '!']
nltk의 WordPunctTokenizer 토큰화:  ['Best', 'of', 'the', 'bunch', 'of', 'all', 'of', 'The', 'wine', 'Cube', 'sku', "'", 's', '.', 'Target', 'Wine', 'Buyer', ':', 'please', 'bring', 'back', '!']
keras의 text_to_word_sequence 토큰화:  ['best', 'of', 'the', 'bunch', 'of', 'all', 'of', 'the', 'wine', 'cube', "sku's", 'target', 'wine', 'buyer', 'please', 'bring', 'back']


#### **위 3개의 토큰화를 하였을 때 차이점을 비교해서 적어보세요**
3개의 토큰화의 다른 점:

In [18]:
# 품사 태깅
tokens = word_tokenize(text)
print('단어 토큰화: ', tokens)
print('pos_tag 품사 태깅:', pos_tag(tokens))#your code)

단어 토큰화:  ['Best', 'of', 'the', 'bunch', 'of', 'all', 'of', 'The', 'wine', 'Cube', 'sku', "'s", '.', 'Target', 'Wine', 'Buyer', ':', 'please', 'bring', 'back', '!']
pos_tag 품사 태깅: [('Best', 'JJS'), ('of', 'IN'), ('the', 'DT'), ('bunch', 'NN'), ('of', 'IN'), ('all', 'DT'), ('of', 'IN'), ('The', 'DT'), ('wine', 'NN'), ('Cube', 'NNP'), ('sku', 'NN'), ("'s", 'POS'), ('.', '.'), ('Target', 'NNP'), ('Wine', 'NNP'), ('Buyer', 'NNP'), (':', ':'), ('please', 'NN'), ('bring', 'VB'), ('back', 'RB'), ('!', '.')]


In [31]:
# 정규 표현식을 사용한 토큰화
re_tokenizer1 = RegexpTokenizer("\s+", gaps=True)#your code) # 공백을 기준으로 토큰화 진행하는 정규 표현식 작성

print(re.findall('\d+', text)) # # 숫자만 찾아내서 리스트로 리턴
print(re_tokenizer1.tokenize(text))

[]
['Best', 'of', 'the', 'bunch', 'of', 'all', 'of', 'The', 'wine', 'Cube', "sku's.", 'Target', 'Wine', 'Buyer:', 'please', 'bring', 'back!']


In [20]:
# 정수 인코딩을 위한 텍스트 전처리
sentences = sent_tokenize(text)
vocab = {}
preprocessed_sentences = []
stop_words = set(stopwords.words('english'))

for sentence in sentences:
  tokenized_sentence1 = word_tokenize(sentence)
  result_1 = []

  for word in tokenized_sentence1:
    word = word.lower()#your code # 모두 소문자로 바꾸기
    if word not in stop_words:
      if len(word) > 2:
        result_1.append(word)
        if word not in vocab:
          vocab[word] = 0
        vocab[word] += 1
  preprocessed_sentences.append(result_1)

vocab_sorted = sorted(vocab.items(), key=lambda x:x[1], reverse = True)
print(vocab_sorted)

[('wine', 2), ('best', 1), ('bunch', 1), ('cube', 1), ('sku', 1), ('target', 1), ('buyer', 1), ('please', 1), ('bring', 1), ('back', 1)]


In [39]:
word_to_index = {}
i = 0
for (word, frequency) in vocab_sorted:
  if frequency > 1:
    i = i+1
    word_to_index[word] = i
print(word_to_index)

{'wine': 1}


In [40]:
word_to_index['SEONG JUN'] = len(word_to_index) + 1 # KHUDA 대신 원하는 단어를 넣어보세요.
print(word_to_index)

{'wine': 1, 'SEONG JUN': 2}


In [41]:
# 단어들을 맵핑되는 정수로 인코딩
encoded_sentences = []
for sentence in preprocessed_sentences:
  encoded_sentence = []
  for word in sentence:
    try:
      encoded_sentence.append(word_to_index[word])
    except KeyError:
      encoded_sentence.append(word_to_index['SEONG JUN'])
  encoded_sentences.append(encoded_sentence)
print(encoded_sentences)

[[2, 2, 1, 2, 2], [2, 1, 2, 2, 2, 2]]


In [24]:
# numpy zero padding
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer2 = Tokenizer()
tokenizer2.fit_on_texts(preprocessed_sentences)
np_encoded = tokenizer2.texts_to_sequences(preprocessed_sentences)

max_len = max_len = max(len(item) for item in np_encoded)#your code
print('최대 길이: ', max_len)

최대 길이:  6


In [25]:
for sentence in np_encoded:
  while len(sentence) < max_len:
    sentence.append(0)

padded_np = np.array(np_encoded)
padded_np

array([[ 2,  3,  1,  4,  5,  0],
       [ 6,  1,  7,  8,  9, 10]])

In [26]:
# keras padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
encoded = tokenizer2.texts_to_sequences(preprocessed_sentences)
padded = pad_sequences(encoded, padding='post') # 패딩할 때 뒤에 0을 붙여 길이를 맞춘다.
padded

array([[ 2,  3,  1,  4,  5,  0],
       [ 6,  1,  7,  8,  9, 10]], dtype=int32)

In [27]:
# one-hot encoding
 
okt = Okt()
my_tokens = okt.morphs("잡 가고싶다")#your code("최진우 교수님은 곽튜브를 닮았다. 살이 찌기 전에는 릴보이를 닮으셨다고 한다.") # 원하는 문장을 넣어보세요.

word_to_index = {word:index for index, word in enumerate(my_tokens)}
print('단어 집합: ', word_to_index)

단어 집합:  {'잡': 0, '가고싶다': 1}


In [28]:
# one-hot encoding 함수
def one_hot_encoding(word, word_to_index):
  one_hot_vector = [0]*(len(word_to_index))
  index = word_to_index[word]
  one_hot_vector[index] = 1
  return one_hot_vector

one_hot_encoding('잡', word_to_index)

[1, 0]

#### **one-hot encoding의 장점과 한계점을 적어보세요.**
*   one-hot encoding의 장점:
1. 단어를 숫자로 바꿔준다.
2. 데이터 변환 과정에서 손실이 거의 없다
*   one-hot encoding의 한계점:
1. 표현 방식은 단어의 개수가 늘어날 수록, 벡터를 저장하기 위해 필요한 공간이 계속 늘어난다는 단점이 있습니다
2. 단어의 유사도를 알 수 없다